# Ollama 3.2 Model 

In [3]:
import requests
import json  # Correct import for JSON parsing

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"  # Ensure this matches the installed model name

# Payload
messages = [{"role": "user", "content": "Describe some of the business applications for sales"}]
payload = {"model": MODEL, "messages": messages}

# API Request
try:
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)  # Enable streaming

    if response.status_code == 200:
        try:
            # Read and process the streamed response line by line
            for line in response.iter_lines():
                if line:
                    response_json = json.loads(line.decode("utf-8"))  # Proper JSON parsing
                    if "message" in response_json:
                        print(response_json["message"]["content"], end="")  # Print response progressively
            print()  # Print newline at the end
        except json.JSONDecodeError:
            print("Failed to parse JSON response.")
            print("Raw Response:", response.text)
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


Sales is a crucial function in any organization, and its applications extend far beyond just generating revenue. Here are some common business applications of sales:

1. **Revenue Generation**: The most obvious application of sales is to generate revenue for an organization by selling products or services.
2. **Market Research**: Sales teams play a vital role in gathering market intelligence, understanding customer needs, and identifying new business opportunities.
3. **Product Development**: Sales data can inform product development, helping companies create products that meet specific customer needs and preferences.
4. **Customer Acquisition and Retention**: Sales efforts focus on acquiring new customers and retaining existing ones, which is essential for long-term business growth.
5. **Competitive Intelligence**: Sales teams gather information about competitors' strategies, pricing, and offerings, enabling organizations to stay competitive in the market.
6. **Market Expansion**: Suc

# Web Scrapping Using Ollama 3.2

In [17]:
import requests
from bs4 import BeautifulSoup # Library to parse HTML

# URL of the web page you want to scrape
URL = "https://scikit-learn.org/stable/" # Replace with the actual URL

# Send a GET request to the webpage
try:
    response = requests.get(URL)

    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

             # Extract specific content (for example, all paragraphs)
            paragraphs = soup.find_all('p') # Find all <p> tags (can be any HTML element)

             # Print the content of each paragraph
            for para in paragraphs:
                print(para.get_text()) # Extract and print text from each paragraph

        except Exception as e :
            print("An error occurred while parsing the content.")
            print(f"Error details:{e}")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        
except requests.exception.RequestsException as e:
    print(f"AN error occurred: {e}")

Identifying which category an object belongs to.

Applications: Spam detection, image recognition.
Algorithms:
Gradient boosting,
            nearest neighbors,
            random forest,
            logistic regression,
            and more...

Predicting a continuous-valued attribute associated with an object.

Applications: Drug response, stock prices.
Algorithms:
Gradient boosting,
            nearest neighbors,
            random forest,
            ridge,
            and more...

Automatic grouping of similar objects into sets.

Applications: Customer segmentation, grouping experiment outcomes.
Algorithms:
k-Means,
            HDBSCAN,
            hierarchical clustering,
            and more...

Reducing the number of random variables to consider.

Applications: Visualization, increased efficiency.
Algorithms:
PCA,
            feature selection,
            non-negative matrix factorization,
            and more...

Comparing, validating and choosing parameters and models.

Appl

# Agentic 1 (Ollama Based)

In [11]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import logging
import re

# Initialize logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
load_dotenv(override=True)

MODEL = 'llama3'  # Ollama model name (Change this to the model you have downloaded e.g., mistral)
API_ENDPOINT = "http://localhost:11434/generate" # Ollama default api endpoint

# Define headers to mimic a browser request
headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# A class to represent a Webpage
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        try:
             response = requests.get(url, headers=headers, timeout=10)
             response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        except requests.exceptions.RequestException as e:
            logging.error(f"Error fetching URL {url}: {e}")
            self.body = ""
            self.title = "Error fetching page"
            self.text = "Error fetching page"
            self.links = []
            return
        
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

# System prompt for generating the brochure
system_prompt = (
    "You are an assistant that analyzes the contents of several relevant pages from a company website "
    "and creates a short brochure about the company for prospective customers, investors and recruits. "
    "Respond in markdown. Include details of company culture, customers and careers/jobs if you have the information."
)

# Generate user prompt for extracting relevant information
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL and type of link in JSON format. "
    user_prompt += "The valid types are: 'company-related' or 'career-related'. "
    user_prompt += "Do not include Terms of Service, Privacy, email links. Respond in the JSON format:"
    user_prompt += "`[{\"url\": \"https://...\", \"type\": \"company-related\"},{\"url\": \"https://...\", \"type\": \"career-related\"}]`\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

def get_links(url):
    website = Website(url)
    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
    ]
    data = {
        "model": MODEL,
        "prompt": "\n".join(message["content"] for message in messages),
        "stream": False # set to false to avoid handling a stream of responses
      }
    response = None # Initialize response variable
    try:
        response = requests.post(API_ENDPOINT, headers=headers, json=data, timeout=20) # increased the timeout to 20 seconds
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        json_response = response.json()
        if 'response' in json_response:
             result = json_response["response"]
             # Remove backticks and code block markers from the generated JSON
             result = re.sub(r'```(json)?', '', result).strip()
             return json.loads(result)
    except requests.exceptions.RequestException as e:
         logging.error(f"Request error: {e}")
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        logging.error(f"JSON Decode Error: {e}. Response Text: {response.text if response else 'No response received'}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

    if response:
        print(f"Error {response.status_code}: {response.text}")
    else:
         print(f"Error: No response received")
    return {"links": []}


# Function to get all details and content of the website
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()  # Use Website(url) here
    links = get_links(url)
    print("Found links:", links)
    for link in links:
       try:
           if isinstance(link, dict) and "url" in link:
             result += f"\n\n{link.get('type', 'unknown-type')}\n"
             result += Website(link["url"]).get_contents() # use Website() here
           else:
              logging.warning(f"Invalid link format: {link}")
       except Exception as e:
            logging.error(f"Error processing link {link}: {e}")
    return result

# User prompt to build the brochure
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt

# Final function to create the brochure
def create_brochure(company_name, url):
     messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ]
     data = {
        "model": MODEL,
        "prompt": "\n".join(message["content"] for message in messages),
        "stream": True # Set to true to receive a stream of responses
      }
     response = None # Initialize response variable
     try:
         response = requests.post(API_ENDPOINT, headers=headers, json=data, stream=True, timeout=20)
         response.raise_for_status()
         full_response = ""
         for line in response.iter_lines():
                if line:
                   json_line = json.loads(line)
                   if 'response' in json_line:
                         full_response += json_line["response"]
                         # Print the response as it streams in
         print(full_response)
         return
     except requests.exceptions.RequestException as e:
        logging.error(f"Request error: {e}")
     except (json.JSONDecodeError, KeyError, TypeError) as e:
        logging.error(f"JSON Decode Error: {e}. Response Text: {response.text if response else 'No response received'}")
     except Exception as e:
         logging.error(f"An unexpected error occurred: {e}")

     if response:
        print(f"Error {response.status_code}: {response.text}")
     else:
         print(f"Error: No response received")
     return

In [12]:
create_brochure("ScikitLearn", "https://scikit-learn.org/stable")

2025-01-31 12:06:06,470 - ERROR - Request error: 404 Client Error: Not Found for url: http://localhost:11434/generate


Error: No response received
Found links: {'links': []}


2025-01-31 12:06:08,525 - ERROR - Request error: 404 Client Error: Not Found for url: http://localhost:11434/generate


Error: No response received
